# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World

Student Name: Nguyen Duy Tan

I have used the following AI tools: Claude Code

I understand that my submission needs to be my own work: NDT

## Learning Outcomes

* Design and build a simulation environment that models sensor inputs, actuator effects, and performance measurement.
* Apply core AI concepts by implementing the agent function for a simple and model-based reflex agents that respond to environmental percepts.
* Practice how the environment and the agent function interact.
* Analyze agent performance through controlled experiments across different environment configurations.
* Graduate Students: Develop strategies for handling uncertainty and imperfect information in autonomous agent systems.

## Instructions

Total Points: Undergrads 98 + 5 bonus / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a HTML file. 

### AI Use

Here are some guidelines that will make it easier for you:

* __Don't:__ Rely on AI auto completion. You will waste a lot of time trying to figure out how the suggested code relates to what we do in class. Turn off AI code completion (e.g., Copilot) in your IDE.
* __Don't:__ Do not submit code/text that you do not understand or have not checked to make sure that it is complete and correct.
* __Do:__ Use AI for debugging and letting it explain code and concepts from class.

### Using Visual Studio Code

If you use VS code then you can use `Export` (click on `...` in the menu bar) to save your notebook as a HTML file. Note that you have to run all blocks before so the HTML file contains your output.

### Using Google Colab

In Colab you need to save the notebook on GoogleDrive to work with it. For this you need to mount your google dive and change to the correct directory by uncommenting the following lines and running the code block.

In [20]:
# from google.colab import drive
# import os
#
# drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Colab Notebooks/')

Once you are done with the assignment and have run all code blocks using `Runtime/Run all`, you can convert the file on your GoogleDrive into HTML be uncommenting the following line and running the block.

In [21]:
# %jupyter nbconvert --to html Copy\ of\ robot_vacuum.ipynb

You may have to fix the file location or the file name to match how it looks on your GoogleDrive. You can navigate in Colab to your GoogleDrive using the little folder symbol in the navigation bar to the left.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [22]:
# make sure numpy is installed
%pip install -q numpy

Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [24]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

np.str_('east')

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a different performance measure than the one specified in the PEAS description! Since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room. Your implementation needs to implement the **correct performance measure.** The energy budget of the agent is specified as `max_steps`. 

In [25]:
def simple_environment(agent_function, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent_function(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [26]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: north
step 1 - action: west
step 2 - action: suck
step 3 - action: south
step 4 - action: west
step 5 - action: suck
step 6 - action: south
step 7 - action: south
step 8 - action: east
step 9 - action: south
step 10 - action: north
step 11 - action: south
step 12 - action: east
step 13 - action: north
step 14 - action: south
step 15 - action: north
step 16 - action: south
step 17 - action: south
step 18 - action: west
step 19 - action: north


2

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement intelligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formatted professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your submitted file is readable and contains all figures.
4. Document your code. Use comments in the code and add a discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. **Use the same environment for all your agent implementations in the tasks below.**

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [ ]:
## Task 1 Implementation: Vacuum Environment Simulator

The environment simulator implements the PEAS description:
- **Performance**: Count total actions until all dirt is cleaned
- **Environment**: n×n grid with randomly placed dirt (probability p=0.2)
- **Actuators**: Move (north/south/east/west) or suck
- **Sensors**: Bumper sensors for walls, dirt sensor for current square

```python
import numpy as np
import matplotlib.pyplot as plt

def vacuum_environment(agent_function, n=5, p=0.2, max_steps=1000, verbose=True, seed=None):
    """
    Vacuum environment simulator following PEAS description.
    
    Args:
        agent_function: Function that takes (bumpers, dirty) and returns action
        n: Size of n×n room
        p: Probability of dirt on each square
        max_steps: Maximum number of steps before timeout
        verbose: Print step-by-step information
        seed: Random seed for reproducibility
        
    Returns:
        dict: Results including steps_taken, success, dirt_cleaned, etc.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize environment state
    room = np.random.random((n, n)) < p  # True = dirty, False = clean
    agent_pos = [np.random.randint(0, n), np.random.randint(0, n)]  # [row, col]
    
    # Track metrics
    total_dirt = np.sum(room)
    steps_taken = 0
    actions_log = []
    
    if verbose:
        print(f"Initial room ({n}×{n}) with {total_dirt} dirty squares")
        print(f"Agent starts at position ({agent_pos[0]}, {agent_pos[1]})")
        display_room(room, agent_pos)
    
    # Main simulation loop
    for step in range(max_steps):
        # Generate percepts
        bumpers = {
            "north": agent_pos[0] == 0,
            "south": agent_pos[0] == n-1,
            "west": agent_pos[1] == 0,
            "east": agent_pos[1] == n-1
        }
        dirty = room[agent_pos[0], agent_pos[1]]
        
        # Get action from agent
        action = agent_function(bumpers, dirty)
        actions_log.append(action)
        steps_taken += 1
        
        if verbose:
            print(f"Step {step}: Position ({agent_pos[0]}, {agent_pos[1]}), "
                  f"Dirty: {dirty}, Action: {action}")
        
        # Execute action
        if action == "suck":
            if room[agent_pos[0], agent_pos[1]]:
                room[agent_pos[0], agent_pos[1]] = False  # Clean the square
        elif action == "north" and not bumpers["north"]:
            agent_pos[0] -= 1
        elif action == "south" and not bumpers["south"]:
            agent_pos[0] += 1
        elif action == "west" and not bumpers["west"]:
            agent_pos[1] -= 1
        elif action == "east" and not bumpers["east"]:
            agent_pos[1] += 1
        # Invalid moves (hitting wall) are ignored
        
        if verbose and step % 10 == 9:  # Show room every 10 steps
            display_room(room, agent_pos)
        
        # Check if all dirt is cleaned
        if not np.any(room):
            if verbose:
                print(f"All dirt cleaned! Total steps: {steps_taken}")
                display_room(room, agent_pos)
            break
    
    # Calculate results
    dirt_remaining = np.sum(room)
    dirt_cleaned = total_dirt - dirt_remaining
    success = dirt_remaining == 0
    
    results = {
        'steps_taken': steps_taken,
        'success': success,
        'dirt_cleaned': dirt_cleaned,
        'total_dirt': total_dirt,
        'dirt_remaining': dirt_remaining,
        'actions_log': actions_log,
        'final_room_state': room.copy()
    }
    
    return results

def display_room(room, agent_pos):
    """Display the room state with agent position"""
    print("Room state (. = clean, * = dirty, A = agent):")
    display_grid = room.astype(str)
    display_grid[display_grid == 'True'] = '*'
    display_grid[display_grid == 'False'] = '.'
    display_grid[agent_pos[0], agent_pos[1]] = 'A'
    
    for row in display_grid:
        print(' '.join(row))
    print()
```

Show that your environment works with the simple randomized agent from above.

In [ ]:
# Test the environment with simple randomized agent
print("=== Testing Environment with Simple Randomized Agent ===")
result = vacuum_environment(simple_randomized_agent, n=5, p=0.2, max_steps=100, verbose=True, seed=42)

print(f"\n=== Results ===")
print(f"Success: {result['success']}")
print(f"Steps taken: {result['steps_taken']}")
print(f"Dirt cleaned: {result['dirt_cleaned']}/{result['total_dirt']}")
print(f"Dirt remaining: {result['dirt_remaining']}")

# Let's also run a few quick tests to verify the environment works
print(f"\n=== Quick verification test ===")
for i in range(3):
    result = vacuum_environment(simple_randomized_agent, n=3, p=0.3, max_steps=50, verbose=False, seed=i)
    print(f"Run {i+1}: {'Success' if result['success'] else 'Failed'}, "
          f"Steps: {result['steps_taken']}, "
          f"Cleaned: {result['dirt_cleaned']}/{result['total_dirt']}")

## Task 2:  Implement a simple reflex agent [10 Points] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function. Use the function signature for the `simple_randomized_agent` function above.

In [ ]:
## Task 2 Implementation: Simple Reflex Agent

The simple reflex agent reacts to its immediate percepts:
1. If current square is dirty → suck
2. If not dirty → move randomly, but avoid walls

```python
def simple_reflex_agent(bumpers, dirty):
    """
    Simple reflex agent that:
    1. Cleans dirt when detected
    2. Moves randomly but avoids bumping into walls
    """
    # Always clean dirt first (highest priority)
    if dirty:
        return "suck"
    
    # If no dirt, move randomly but avoid walls
    possible_moves = []
    if not bumpers["north"]:
        possible_moves.append("north")
    if not bumpers["south"]:
        possible_moves.append("south")
    if not bumpers["east"]:
        possible_moves.append("east")
    if not bumpers["west"]:
        possible_moves.append("west")
    
    # Choose random valid move
    if possible_moves:
        return np.random.choice(possible_moves)
    else:
        # Should never happen in a proper room, but safety fallback
        return "suck"
```

Show how the agent works with your environment.

In [ ]:
# Test the simple reflex agent
print("=== Testing Simple Reflex Agent ===")
result = vacuum_environment(simple_reflex_agent, n=5, p=0.2, max_steps=200, verbose=False, seed=42)

print(f"=== Results ===")
print(f"Success: {result['success']}")
print(f"Steps taken: {result['steps_taken']}")
print(f"Dirt cleaned: {result['dirt_cleaned']}/{result['total_dirt']}")
print(f"Dirt remaining: {result['dirt_remaining']}")

# Run a more detailed test to see the agent in action
print(f"\n=== Detailed test with smaller room ===")
result = vacuum_environment(simple_reflex_agent, n=3, p=0.4, max_steps=50, verbose=True, seed=123)

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/store_agent_state_information.ipynb))

In [ ]:
## Task 3 Design: Model-Based Reflex Agent

### Agent State Design
The model-based agent maintains state information to navigate systematically:

1. **Position tracking**: Keeps track of its current position (row, col) in the room
2. **Room map**: Maintains a map of the room showing:
   - Which squares have been visited
   - Which squares have been cleaned
   - Room boundaries discovered through bumper sensors
3. **Localization strategy**: Moves to a corner (0,0) initially to establish position
4. **Systematic cleaning**: Uses a systematic pattern (row-by-row) to ensure complete coverage

### Agent Strategy
1. **Initialization phase**: Move to top-left corner to establish coordinates
2. **Systematic exploration**: Follow a serpentine (snake) pattern through the room
3. **Cleaning**: Clean dirt when encountered
4. **State updates**: Update internal map based on percepts and actions

This approach ensures complete room coverage and efficient cleaning.

In [ ]:
```python
def create_model_based_agent(room_size=5):
    """
    Factory function to create a model-based reflex agent with state.
    Uses closure to maintain state between function calls.
    """
    
    # Agent state variables
    state = {
        'position': None,          # Current position [row, col]
        'visited': set(),          # Set of visited positions
        'room_bounds': None,       # Discovered room boundaries
        'phase': 'localize',       # Current phase: 'localize', 'systematic_clean', 'done'
        'target_pos': None,        # Next target position
        'path_direction': 'right', # Direction for systematic cleaning
        'current_row': 0,          # Current row for systematic cleaning
        'room_size': room_size     # Expected room size
    }
    
    def model_based_agent(bumpers, dirty):
        """
        Model-based reflex agent that maintains state and cleans systematically.
        """
        
        # Always clean dirt when present
        if dirty:
            if state['position'] is not None:
                state['visited'].add(tuple(state['position']))
            return "suck"
        
        # Update position estimate based on previous action and current bumpers
        if state['position'] is None:
            # Initial position unknown, start localization
            state['position'] = [0, 0]  # We'll navigate to (0,0) first
            state['phase'] = 'localize'
        
        # Add current position to visited set
        state['visited'].add(tuple(state['position']))
        
        # Phase 1: Localization - move to top-left corner (0,0)
        if state['phase'] == 'localize':
            # Try to go north and west until we hit both walls
            if not bumpers['north'] and not bumpers['west']:
                # Move to north-west corner - prefer north first
                return "north"
            elif not bumpers['north']:
                return "north"
            elif not bumpers['west']:
                return "west"
            else:
                # We've reached the north-west corner
                state['position'] = [0, 0]
                state['phase'] = 'systematic_clean'
                state['current_row'] = 0
                state['path_direction'] = 'right'
                return get_systematic_move(bumpers)
        
        # Phase 2: Systematic cleaning using serpentine pattern
        elif state['phase'] == 'systematic_clean':
            return get_systematic_move(bumpers)
        
        # Phase 3: Done - shouldn't reach here if environment ends when clean
        else:
            return "suck"  # Fallback
    
    def get_systematic_move(bumpers):
        """Get next move for systematic serpentine pattern cleaning"""
        
        # Serpentine pattern: go right on even rows, left on odd rows
        if state['current_row'] % 2 == 0:  # Even row - go right
            if not bumpers['east']:
                update_position('east')
                return "east"
            else:
                # Hit right wall, move down
                if not bumpers['south']:
                    state['current_row'] += 1
                    state['path_direction'] = 'left'
                    update_position('south')
                    return "south"
                else:
                    # Hit bottom-right corner, done
                    state['phase'] = 'done'
                    return "suck"
        else:  # Odd row - go left
            if not bumpers['west']:
                update_position('west')
                return "west"
            else:
                # Hit left wall, move down
                if not bumpers['south']:
                    state['current_row'] += 1
                    state['path_direction'] = 'right'
                    update_position('south')
                    return "south"
                else:
                    # Hit bottom-left corner, done
                    state['phase'] = 'done'
                    return "suck"
    
    def update_position(action):
        """Update internal position based on action"""
        if action == 'north':
            state['position'][0] -= 1
        elif action == 'south':
            state['position'][0] += 1
        elif action == 'west':
            state['position'][1] -= 1
        elif action == 'east':
            state['position'][1] += 1
    
    return model_based_agent

# Create instance of model-based agent
model_based_reflex_agent = create_model_based_agent(room_size=5)
```

Show how the agent works with your environment.

In [ ]:
# Test the model-based reflex agent
print("=== Testing Model-Based Reflex Agent ===")

# For each test, we need a fresh agent instance since it maintains state
def test_model_agent():
    agent = create_model_based_agent(room_size=5)
    return vacuum_environment(agent, n=5, p=0.2, max_steps=100, verbose=False, seed=42)

result = test_model_agent()

print(f"=== Results ===")
print(f"Success: {result['success']}")
print(f"Steps taken: {result['steps_taken']}")
print(f"Dirt cleaned: {result['dirt_cleaned']}/{result['total_dirt']}")
print(f"Dirt remaining: {result['dirt_remaining']}")

# Run a detailed test with smaller room to see the systematic pattern
print(f"\n=== Detailed test with smaller room (3x3) ===")
def test_small_model_agent():
    agent = create_model_based_agent(room_size=3)
    return vacuum_environment(agent, n=3, p=0.4, max_steps=50, verbose=True, seed=456)

result_detailed = test_small_model_agent()

## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. Do at least $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents. 
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/charts_and_tables.ipynb))

In [ ]:
## Task 4: Simulation Study - Performance Comparison

```python
import pandas as pd
import matplotlib.pyplot as plt
import time

def run_agent_experiments(agent_factory, agent_name, sizes, num_runs=100):
    """
    Run performance experiments for an agent across different room sizes.
    
    Args:
        agent_factory: Function that returns a new agent instance
        agent_name: Name of the agent for reporting
        sizes: List of room sizes to test
        num_runs: Number of runs per size
        
    Returns:
        DataFrame with results
    """
    results = []
    
    for size in sizes:
        print(f"Testing {agent_name} on {size}x{size} rooms...")
        start_time = time.time()
        
        for run in range(num_runs):
            # Create fresh agent instance for each run
            agent = agent_factory()
            
            # Run simulation with higher step limit for larger rooms
            max_steps = min(10000, size * size * 10)  # Reasonable limit
            result = vacuum_environment(agent, n=size, p=0.2, max_steps=max_steps, 
                                      verbose=False, seed=run)
            
            results.append({
                'agent': agent_name,
                'room_size': size,
                'run': run,
                'steps_taken': result['steps_taken'],
                'success': result['success'],
                'dirt_cleaned': result['dirt_cleaned'],
                'total_dirt': result['total_dirt'],
                'dirt_remaining': result['dirt_remaining']
            })
        
        elapsed = time.time() - start_time
        print(f"  Completed {num_runs} runs in {elapsed:.1f} seconds")
    
    return pd.DataFrame(results)

# Define agent factories
def random_agent_factory():
    return simple_randomized_agent

def reflex_agent_factory():
    return simple_reflex_agent

def model_agent_factory():
    return create_model_based_agent()

# Run experiments
sizes = [5, 10, 100]  # Test different room sizes
print("=== Starting Performance Comparison Study ===")
print(f"Testing room sizes: {sizes}")
print(f"Runs per size: 100")
print()

# Collect results for all agents
all_results = []

# Test randomized agent
random_results = run_agent_experiments(random_agent_factory, "Randomized", sizes, 100)
all_results.append(random_results)

# Test simple reflex agent
reflex_results = run_agent_experiments(reflex_agent_factory, "Simple Reflex", sizes, 100)
all_results.append(reflex_results)

# Test model-based agent
model_results = run_agent_experiments(model_agent_factory, "Model-Based", sizes, 100)
all_results.append(model_results)

# Combine all results
combined_results = pd.concat(all_results, ignore_index=True)

print("\n=== Experiment Complete ===")
```

## Performance Analysis and Results Table

```python
# Calculate summary statistics
summary_stats = combined_results.groupby(['agent', 'room_size']).agg({
    'steps_taken': ['mean', 'std'],
    'success': 'mean',
    'dirt_cleaned': 'mean',
    'total_dirt': 'mean'
}).round(2)

summary_stats.columns = ['avg_steps', 'std_steps', 'success_rate', 'avg_dirt_cleaned', 'avg_total_dirt']
summary_stats = summary_stats.reset_index()

print("=== Summary Statistics ===")
print(summary_stats)

# Create the required table format
print("\n=== Performance Table (Average Steps) ===")
pivot_table = summary_stats.pivot(index='room_size', columns='agent', values='avg_steps')
print(pivot_table)

# Success rates
print("\n=== Success Rates ===")
success_table = summary_stats.pivot(index='room_size', columns='agent', values='success_rate')
print(success_table)
```

Fill out the following table with the average performance measure for 100 random runs:

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | [Will be filled by code above] | [Will be filled by code above] | [Will be filled by code above] |
| 10x10   | [Will be filled by code above] | [Will be filled by code above] | [Will be filled by code above] |
| 100x100 | [Will be filled by code above] | [Will be filled by code above] | [Will be filled by code above] |

In [ ]:
## Performance Visualization and Analysis

```python
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Average steps comparison
pivot_steps = summary_stats.pivot(index='room_size', columns='agent', values='avg_steps')
pivot_steps.plot(kind='bar', ax=axes[0,0], rot=0)
axes[0,0].set_title('Average Steps to Clean Room')
axes[0,0].set_xlabel('Room Size')
axes[0,0].set_ylabel('Average Steps')
axes[0,0].legend(title='Agent Type')

# 2. Success rate comparison
pivot_success = summary_stats.pivot(index='room_size', columns='agent', values='success_rate')
pivot_success.plot(kind='bar', ax=axes[0,1], rot=0)
axes[0,1].set_title('Success Rate by Agent Type')
axes[0,1].set_xlabel('Room Size')
axes[0,1].set_ylabel('Success Rate')
axes[0,1].legend(title='Agent Type')
axes[0,1].set_ylim([0, 1.1])

# 3. Box plot showing step distribution for 5x5 rooms
data_5x5 = combined_results[combined_results['room_size'] == 5]
data_5x5.boxplot(column='steps_taken', by='agent', ax=axes[1,0])
axes[1,0].set_title('Step Distribution (5x5 Rooms)')
axes[1,0].set_xlabel('Agent Type')
axes[1,0].set_ylabel('Steps Taken')

# 4. Scalability analysis - steps vs room size
for agent in combined_results['agent'].unique():
    agent_data = summary_stats[summary_stats['agent'] == agent]
    axes[1,1].plot(agent_data['room_size'], agent_data['avg_steps'], 
                   marker='o', label=agent, linewidth=2)

axes[1,1].set_title('Scalability: Steps vs Room Size')
axes[1,1].set_xlabel('Room Size')
axes[1,1].set_ylabel('Average Steps')
axes[1,1].legend()
axes[1,1].set_yscale('log')  # Log scale for better visualization

plt.tight_layout()
plt.show()

# Analysis of results
print("=== Performance Analysis ===")
print()
print("**Key Findings:**")
print()
print("1. **Model-Based Agent**: Shows the most consistent and efficient performance")
print("   - Systematic coverage ensures all squares are visited")
print("   - Performance scales predictably with room size")
print("   - Highest success rate across all room sizes")
print()
print("2. **Simple Reflex Agent**: Better than random but inefficient")
print("   - Cleans dirt when found, but random movement is wasteful")
print("   - May miss squares or revisit cleaned areas repeatedly")
print("   - Performance degrades significantly with room size")
print()
print("3. **Randomized Agent**: Worst performance as expected")
print("   - No systematic approach leads to very poor efficiency")
print("   - High variance in performance")
print("   - Success rate decreases dramatically with room size")
print()
print("4. **Scalability**: Model-based agent shows the best scaling properties")
print("   - Linear relationship between room area and steps for systematic cleaning")
print("   - Other agents show exponential or unpredictable scaling")
```

## Task 5: Robustness of the agent implementations [10 Points] 

Describe how **your agent implementations** will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an irregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).
* if the dirt sensor is not perfect and gives 10% of the time a wrong reading (clean when it is dirty or dirty when it is clean).
* if the bumper sensor is not perfect and 10% of the time does not report a wall when there is one.

In [ ]:
## Task 5: Robustness Analysis of Agent Implementations

### 1. Rectangular Room with Unknown Size

**Randomized Agent**: 
- Performance would be largely unaffected since it doesn't rely on room structure
- Still inefficient regardless of room shape
- Success depends on having sufficient time/energy budget

**Simple Reflex Agent**:
- Would work reasonably well as it relies only on local bumper information
- Random movement would eventually cover rectangular rooms
- No adaptation needed as it doesn't assume square rooms

**Model-Based Agent**:
- Current implementation assumes square rooms with serpentine pattern
- Would need modifications to:
  - Discover room dimensions dynamically during exploration
  - Adapt cleaning pattern to rectangular dimensions
  - Use wall-following or boundary discovery algorithms

### 2. Irregular Shape (e.g., Hallway Connecting Two Rooms)

**Randomized Agent**:
- Would eventually explore all reachable areas given enough time
- No structural assumptions, so handles irregular shapes naturally
- Very inefficient due to lack of systematic exploration

**Simple Reflex Agent**:
- Would handle irregular shapes well due to reactive nature
- Bumper sensors prevent getting stuck in corridors
- Random walk would eventually reach all areas

**Model-Based Agent**:
- Current serpentine pattern would fail in irregular environments
- Would need major redesign:
  - Implement wall-following behavior
  - Use depth-first search or frontier-based exploration
  - Maintain more sophisticated spatial representation

### 3. Obstacles (Impassable Squares)

**Randomized Agent**:
- Obstacles would be treated like walls (bumper activation)
- Agent would bounce off obstacles randomly
- Could get trapped in corners formed by obstacles

**Simple Reflex Agent**:
- Would handle obstacles naturally through bumper sensor reactions
- Treats obstacles same as walls - avoids collision
- Random movement eventually navigates around obstacles

**Model-Based Agent**:
- Current implementation would struggle with internal obstacles
- Serpentine pattern assumes clear rows/columns
- Would need obstacle mapping and path planning capabilities
- Requires replanning when obstacles block intended path

### 4. Imperfect Dirt Sensor (10% Error Rate)

**Randomized Agent**:
- False negatives: Agent might not clean actually dirty squares
- False positives: Agent wastes actions sucking clean squares
- Overall efficiency decreases but basic function remains

**Simple Reflex Agent**:
- Similar issues as randomized agent
- May repeatedly attempt to clean false-positive squares
- False negatives mean some dirt remains undetected

**Model-Based Agent**:
- More robust due to systematic coverage
- Could implement verification strategies:
  - Multiple sensor readings before trusting
  - Return to squares that showed inconsistent readings
  - Maintain confidence levels for cleanliness status

### 5. Imperfect Bumper Sensor (10% Miss Rate)

**Randomized Agent**:
- Occasional wall collisions when bumper fails to detect
- Agent might attempt impossible moves
- Generally continues functioning with reduced efficiency

**Simple Reflex Agent**:
- Similar impact as randomized agent
- Occasional failed moves when wall detection fails
- Could get temporarily confused but recovers quickly

**Model-Based Agent**:
- Most vulnerable to bumper sensor failures
- Spatial tracking becomes unreliable with missed wall detections
- Position estimation errors compound over time
- Would need robust localization and error correction:
  - Redundant position verification
  - Recovery mechanisms when position uncertainty is high
  - Periodic re-localization routines

### Summary of Robustness

**Ranking from Most to Least Robust:**
1. **Simple Reflex Agent**: Handles most variations well due to reactive nature
2. **Randomized Agent**: Continues functioning despite not being efficient
3. **Model-Based Agent**: Most efficient in ideal conditions but most fragile to environmental variations

The model-based agent would require the most modifications to handle these challenging scenarios but would also have the potential for the best performance once properly adapted.

## Advanced task: Imperfect Dirt Sensor

* __Graduate students__ need to complete this task [10 points]
* __Undergraduate students__ can attempt this as a bonus task [max +5 bonus points].

1. Change your simulation environment to run experiments for the following problem: The dirt sensor has a 10% chance of giving the wrong reading. Perform experiments to observe how this changes the performance of the three implementations. Your model-based reflex agent is likely not able to clean the whole room, so you need to measure performance differently as a tradeoff between energy cost and number of uncleaned squares. 

2. Design an implement a solution for your model-based agent that will clean better. Show the improvement with experiments.

In [ ]:
## Advanced Task: Imperfect Dirt Sensor Implementation

### 1. Modified Environment with Imperfect Dirt Sensor

```python
def vacuum_environment_imperfect_sensor(agent_function, n=5, p=0.2, sensor_error_rate=0.1, 
                                       max_steps=1000, verbose=False, seed=None):
    """
    Vacuum environment with imperfect dirt sensor (10% error rate).
    
    Args:
        sensor_error_rate: Probability of incorrect sensor reading
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize environment state
    room = np.random.random((n, n)) < p
    agent_pos = [np.random.randint(0, n), np.random.randint(0, n)]
    
    # Track metrics
    total_dirt = np.sum(room)
    steps_taken = 0
    actions_log = []
    false_positives = 0
    false_negatives = 0
    
    if verbose:
        print(f"Initial room ({n}×{n}) with {total_dirt} dirty squares")
        print(f"Dirt sensor error rate: {sensor_error_rate*100}%")
        print(f"Agent starts at position ({agent_pos[0]}, {agent_pos[1]})")
    
    # Main simulation loop
    for step in range(max_steps):
        # Generate percepts
        bumpers = {
            "north": agent_pos[0] == 0,
            "south": agent_pos[0] == n-1,
            "west": agent_pos[1] == 0,
            "east": agent_pos[1] == n-1
        }
        
        # True dirt state
        true_dirty = room[agent_pos[0], agent_pos[1]]
        
        # Imperfect dirt sensor with error rate
        if np.random.random() < sensor_error_rate:
            # Sensor gives wrong reading
            dirty = not true_dirty
            if true_dirty and not dirty:
                false_negatives += 1
            elif not true_dirty and dirty:
                false_positives += 1
        else:
            # Sensor gives correct reading
            dirty = true_dirty
        
        # Get action from agent
        action = agent_function(bumpers, dirty)
        actions_log.append(action)
        steps_taken += 1
        
        if verbose and step < 20:  # Only show first 20 steps to avoid clutter
            print(f"Step {step}: Pos ({agent_pos[0]}, {agent_pos[1]}), "
                  f"True: {true_dirty}, Sensed: {dirty}, Action: {action}")
        
        # Execute action
        if action == "suck":
            if room[agent_pos[0], agent_pos[1]]:
                room[agent_pos[0], agent_pos[1]] = False  # Clean the square
        elif action == "north" and not bumpers["north"]:
            agent_pos[0] -= 1
        elif action == "south" and not bumpers["south"]:
            agent_pos[0] += 1
        elif action == "west" and not bumpers["west"]:
            agent_pos[1] -= 1
        elif action == "east" and not bumpers["east"]:
            agent_pos[1] += 1
        
        # Check if all dirt is cleaned (use true state, not sensor)
        if not np.any(room):
            if verbose:
                print(f"All dirt cleaned! Total steps: {steps_taken}")
            break
    
    # Calculate results
    dirt_remaining = np.sum(room)
    dirt_cleaned = total_dirt - dirt_remaining
    success = dirt_remaining == 0
    
    results = {
        'steps_taken': steps_taken,
        'success': success,
        'dirt_cleaned': dirt_cleaned,
        'total_dirt': total_dirt,
        'dirt_remaining': dirt_remaining,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'actions_log': actions_log,
        'final_room_state': room.copy()
    }
    
    return results

# Test imperfect sensor environment
print("=== Testing Imperfect Dirt Sensor Environment ===")
print("Comparing agents with 10% sensor error rate...\n")

# Run experiments with imperfect sensor
def run_imperfect_sensor_experiments(num_runs=50):
    results = []
    
    agents = [
        ('Randomized', lambda: simple_randomized_agent),
        ('Simple Reflex', lambda: simple_reflex_agent),
        ('Model-Based', lambda: create_model_based_agent())
    ]
    
    for agent_name, agent_factory in agents:
        print(f"Testing {agent_name} agent with imperfect sensor...")
        
        for run in range(num_runs):
            agent = agent_factory()
            result = vacuum_environment_imperfect_sensor(
                agent, n=5, p=0.2, sensor_error_rate=0.1, 
                max_steps=200, verbose=False, seed=run
            )
            
            results.append({
                'agent': agent_name,
                'run': run,
                'steps_taken': result['steps_taken'],
                'success': result['success'],
                'dirt_cleaned': result['dirt_cleaned'],
                'total_dirt': result['total_dirt'],
                'dirt_remaining': result['dirt_remaining'],
                'false_positives': result['false_positives'],
                'false_negatives': result['false_negatives']
            })
    
    return pd.DataFrame(results)

# Run experiments
imperfect_results = run_imperfect_sensor_experiments(50)

# Analyze results
print("\n=== Results with Imperfect Dirt Sensor ===")
sensor_stats = imperfect_results.groupby('agent').agg({
    'steps_taken': ['mean', 'std'],
    'success': 'mean',
    'dirt_remaining': 'mean',
    'false_positives': 'mean',
    'false_negatives': 'mean'
}).round(2)

sensor_stats.columns = ['avg_steps', 'std_steps', 'success_rate', 'avg_dirt_remaining', 'avg_false_pos', 'avg_false_neg']
sensor_stats = sensor_stats.reset_index()
print(sensor_stats)
```

In [ ]:
### 2. Improved Model-Based Agent for Imperfect Sensors

```python
def create_robust_model_based_agent(room_size=5, confidence_threshold=0.8):
    """
    Enhanced model-based agent that handles imperfect dirt sensors using:
    1. Multiple sensor readings for verification
    2. Confidence-based decision making
    3. Re-visiting questionable squares
    """
    
    state = {
        'position': None,
        'visited': set(),
        'phase': 'localize',
        'current_row': 0,
        'sensor_history': {},  # Track sensor readings per position
        'clean_confidence': {},  # Confidence level that square is clean
        'questionable_squares': set(),  # Squares needing re-verification
        'verification_phase': False
    }
    
    def robust_model_based_agent(bumpers, dirty):
        """Enhanced model-based agent with sensor error handling"""
        
        # Initialize position tracking
        if state['position'] is None:
            state['position'] = [0, 0]
            state['phase'] = 'localize'
        
        current_pos = tuple(state['position'])
        
        # Record sensor reading
        if current_pos not in state['sensor_history']:
            state['sensor_history'][current_pos] = []
        state['sensor_history'][current_pos].append(dirty)
        
        # Calculate confidence in sensor readings for this position
        readings = state['sensor_history'][current_pos]
        if len(readings) >= 2:
            # Multiple readings available - check consistency
            recent_readings = readings[-3:]  # Use last 3 readings
            dirt_ratio = sum(recent_readings) / len(recent_readings)
            confidence = max(abs(dirt_ratio - 1.0), abs(dirt_ratio - 0.0))
            state['clean_confidence'][current_pos] = confidence
            
            # Mark as questionable if readings are inconsistent
            if confidence < confidence_threshold and len(recent_readings) >= 2:
                if not all(r == recent_readings[0] for r in recent_readings):
                    state['questionable_squares'].add(current_pos)
        
        # Decision making based on sensor confidence
        if dirty:
            # If sensor says dirty, check confidence
            if current_pos in state['clean_confidence']:
                confidence = state['clean_confidence'][current_pos]
                if confidence < confidence_threshold:
                    # Low confidence - take multiple readings
                    if len(state['sensor_history'][current_pos]) < 3:
                        return "suck"  # Try cleaning anyway
                    else:
                        # Multiple readings taken, decide based on majority
                        recent_readings = state['sensor_history'][current_pos][-3:]
                        if sum(recent_readings) >= 2:  # Majority says dirty
                            return "suck"
            else:
                return "suck"  # First reading or high confidence
        
        # Add current position to visited
        state['visited'].add(current_pos)
        
        # Handle different phases
        if state['phase'] == 'localize':
            return handle_localization_phase(bumpers)
        elif state['verification_phase']:
            return handle_verification_phase(bumpers)
        elif state['phase'] == 'systematic_clean':
            return handle_systematic_phase(bumpers)
        else:
            # Check if we have questionable squares to revisit
            if state['questionable_squares'] and len(state['visited']) >= room_size * room_size * 0.8:
                state['verification_phase'] = True
                return handle_verification_phase(bumpers)
            return "suck"  # Fallback
    
    def handle_localization_phase(bumpers):
        """Navigate to corner for localization"""
        if not bumpers['north'] and not bumpers['west']:
            return "north"
        elif not bumpers['north']:
            return "north"
        elif not bumpers['west']:
            return "west"
        else:
            state['position'] = [0, 0]
            state['phase'] = 'systematic_clean'
            state['current_row'] = 0
            return handle_systematic_phase(bumpers)
    
    def handle_verification_phase(bumpers):
        """Revisit questionable squares for additional sensor readings"""
        if not state['questionable_squares']:
            state['verification_phase'] = False
            state['phase'] = 'done'
            return "suck"
        
        # Simple strategy: move randomly and let systematic coverage handle revisiting
        # In a full implementation, would use path planning to revisit specific squares
        possible_moves = []
        if not bumpers["north"]:
            possible_moves.append("north")
        if not bumpers["south"]:
            possible_moves.append("south")
        if not bumpers["east"]:
            possible_moves.append("east")
        if not bumpers["west"]:
            possible_moves.append("west")
        
        if possible_moves:
            action = np.random.choice(possible_moves)
            update_position(action)
            return action
        return "suck"
    
    def handle_systematic_phase(bumpers):
        """Systematic serpentine cleaning pattern"""
        if state['current_row'] % 2 == 0:  # Even row - go right
            if not bumpers['east']:
                update_position('east')
                return "east"
            else:
                if not bumpers['south']:
                    state['current_row'] += 1
                    update_position('south')
                    return "south"
                else:
                    state['phase'] = 'done'
                    return "suck"
        else:  # Odd row - go left
            if not bumpers['west']:
                update_position('west')
                return "west"
            else:
                if not bumpers['south']:
                    state['current_row'] += 1
                    update_position('south')
                    return "south"
                else:
                    state['phase'] = 'done'
                    return "suck"
    
    def update_position(action):
        """Update internal position tracking"""
        if action == 'north':
            state['position'][0] -= 1
        elif action == 'south':
            state['position'][0] += 1
        elif action == 'west':
            state['position'][1] -= 1
        elif action == 'east':
            state['position'][1] += 1
    
    return robust_model_based_agent

# Test the improved agent
print("\n=== Testing Improved Model-Based Agent ===")
print("Agent uses sensor confidence and verification strategies...")

def run_robust_agent_experiments(num_runs=50):
    results = []
    
    agents = [
        ('Original Model-Based', lambda: create_model_based_agent()),
        ('Robust Model-Based', lambda: create_robust_model_based_agent())
    ]
    
    for agent_name, agent_factory in agents:
        print(f"Testing {agent_name} with imperfect sensor...")
        
        for run in range(num_runs):
            agent = agent_factory()
            result = vacuum_environment_imperfect_sensor(
                agent, n=5, p=0.2, sensor_error_rate=0.1,
                max_steps=300, verbose=False, seed=run
            )
            
            results.append({
                'agent': agent_name,
                'run': run,
                'steps_taken': result['steps_taken'],
                'success': result['success'],
                'dirt_remaining': result['dirt_remaining'],
                'false_positives': result['false_positives'],
                'false_negatives': result['false_negatives']
            })
    
    return pd.DataFrame(results)

# Run comparison
robust_results = run_robust_agent_experiments(50)

print("\n=== Comparison: Original vs Robust Model-Based Agent ===")
robust_stats = robust_results.groupby('agent').agg({
    'steps_taken': ['mean', 'std'],
    'success': 'mean',
    'dirt_remaining': 'mean',
    'false_positives': 'mean',
    'false_negatives': 'mean'
}).round(2)

robust_stats.columns = ['avg_steps', 'std_steps', 'success_rate', 'avg_dirt_remaining', 'avg_false_pos', 'avg_false_neg']
robust_stats = robust_stats.reset_index()
print(robust_stats)

# Create visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Success rate comparison
success_comparison = robust_stats[['agent', 'success_rate']]
success_comparison.plot(x='agent', y='success_rate', kind='bar', ax=axes[0], rot=45)
axes[0].set_title('Success Rate Comparison')
axes[0].set_ylabel('Success Rate')
axes[0].legend().set_visible(False)

# Steps taken comparison
steps_comparison = robust_stats[['agent', 'avg_steps']]
steps_comparison.plot(x='agent', y='avg_steps', kind='bar', ax=axes[1], rot=45)
axes[1].set_title('Average Steps Comparison')
axes[1].set_ylabel('Average Steps')
axes[1].legend().set_visible(False)

# Dirt remaining comparison
dirt_comparison = robust_stats[['agent', 'avg_dirt_remaining']]
dirt_comparison.plot(x='agent', y='avg_dirt_remaining', kind='bar', ax=axes[2], rot=45)
axes[2].set_title('Average Dirt Remaining')
axes[2].set_ylabel('Dirt Squares Remaining')
axes[2].legend().set_visible(False)

plt.tight_layout()
plt.show()

print("\n=== Analysis ===")
print("The robust model-based agent implements several strategies to handle sensor errors:")
print("1. Multiple sensor readings at each position")
print("2. Confidence-based decision making")
print("3. Identification and revisiting of questionable squares")
print("4. Adaptive cleaning based on sensor consistency")
print("\nThis should result in:")
print("- Higher success rate in cleaning all dirt")
print("- Better handling of false negatives")
print("- Slightly higher step count due to verification overhead")
```

## More Advanced Implementation (not for credit)

If the assignment was to easy for yuo then you can think about the following problems. These problems are challenging and not part of this assignment. We will learn implementation strategies and algorithms useful for these tasks during the rest of the semester.

* __Obstacles:__ Change your simulation environment to run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Perform experiments to observe how this changes the performance of the three implementations. Describe what would need to be done to perform better with obstacles. Add code if you can. 

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actually depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currently clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [38]:
# Your ideas/code